In [1]:
import json
import re
from pprint import pprint as pp
from typing import Set, List, Dict, NamedTuple

In [2]:
class Page(NamedTuple):
    pid: int
    title: str
class Entry(NamedTuple):
    page: Page
    attr: str
    text: str

In [12]:
fn = "/home/ebal/Data/wikipedia/shinra2019/JP-5/annotation/Company_dist.json"
entries = []
with open(fn) as f:
    for line in f:
        data = json.loads(line)
        entries.append(Entry(Page(int(data["page_id"]), data["title"]), data["attribute"], data["text_offset"]["text"]))
allpages = {entry.page for entry in entries}
fn = "/home/ebal/Data/wikipedia/shinra2019/JP-5/EXTRACTED/Company_train.json"
myentries = []
with open(fn) as f:
    reg = re.compile(r"<[^>]*?>")
    for line in f:
        data = json.loads(line)
        myentries.append(Entry(Page(int(data["page_id"]), data["title"]), data["attribute"], reg.sub("", data["html_offset"]["text"])))

In [13]:
def searchr(reg: str, attr: str, pages: Set[Page], entries: Set[Entry]) -> Set[Page]:
    reg = re.compile(reg)
    return {et.page for et in entries if et.page in pages and et.attr == attr and reg.match(et.text)}
def searchf(fun, attr: str, pages: Set[Page], entries: Set[Entry]) -> Set[Page]:
    return {et.page for et in entries if et.page in pages and et.attr == attr and fun(et.text)}

In [14]:
tokyo = searchr(r"東京都?", "本拠地", allpages, entries)
ltd = searchr(r"株式(会社)?", "種類", tokyo, entries)

from show_a_table.model.refiner import date
f1996t2020 = date.DateRangeRefiner("設立年")
f1996t2020._start = "1996-*-*"
f1996t2020._end = "2030-*-*"
exam = f1996t2020._mk_exam()

ans = searchf(exam, "設立年", ltd, entries)

Cannot parse as date ')9月22日' by '%-E%-kO年%m月%d日'


In [15]:
tokyo = searchr(r"東京都?", "本拠地", allpages, myentries)
ltd = searchr(r"株式(会社)?", "種類", tokyo, myentries)
myans = searchf(exam, "設立年", ltd, entries)

In [16]:
myans

{Page(pid=44308, title='近代映画社'),
 Page(pid=728612, title='ぷれっそホールディングス'),
 Page(pid=1423956, title='ジャパンケアサービス')}

In [10]:
ans

{Page(pid=44308, title='近代映画社'),
 Page(pid=47515, title='BSフジ'),
 Page(pid=54730, title='BSジャパン'),
 Page(pid=54734, title='BS日本'),
 Page(pid=58506, title='ニトロプラス'),
 Page(pid=66781, title='アサヒビール'),
 Page(pid=102896, title='日本ミルクコミュニティ'),
 Page(pid=116106, title='ユーシーカード'),
 Page(pid=169693, title='アステラス製薬'),
 Page(pid=170172, title='1-UPスタジオ'),
 Page(pid=170238, title='アニメインターナショナルカンパニー'),
 Page(pid=184198, title='資産管理サービス信託銀行'),
 Page(pid=223719, title='東急ホテルズ'),
 Page(pid=314597, title='日立オムロンターミナルソリューションズ'),
 Page(pid=314872, title='JTBパブリッシング'),
 Page(pid=316692, title='OKWAVE'),
 Page(pid=326320, title='三菱樹脂'),
 Page(pid=377893, title='テクノプロ・ホールディングス'),
 Page(pid=386937, title='AGスクエア'),
 Page(pid=402627, title='エリコネイル'),
 Page(pid=417056, title='ローソン・エイティエム・ネットワークス'),
 Page(pid=463109, title='アイライン_(企業)'),
 Page(pid=464945, title='LINE_Business_Partners'),
 Page(pid=586543, title='AQインタラクティブ'),
 Page(pid=594415, title='サイバーステップ'),
 Page(pid=707807, title='中日本エクシス'),
 Page(pid=71